## Synthetic Site Map Generator

In [ ]:
! pip install diffusers transformers torch opencv-python pillow

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch
from PIL import Image
import cv2
import numpy as np
import os

# Directories for processed site maps and generated synthetic images
processed_dir = "processed_sitemaps"
synthetic_dir = "synthetic_sitemaps_v2"
os.makedirs(synthetic_dir, exist_ok=True)

# Load ControlNet and Stable Diffusion models
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16)
pipe.to("cuda")

# Function to preprocess image with OpenCV Canny edge detection
def preprocess_with_canny(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(image, threshold1=100, threshold2=200)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(edges)

# Generate synthetic site maps
prompt = "A detailed site map with modern architectural elements and a clean layout"
for i, filename in enumerate(os.listdir(processed_dir)):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        edge_image = preprocess_with_canny(os.path.join(processed_dir, filename))

        edge_image = np.array(edge_image).astype(np.float32) / 255.0
        edge_image = torch.tensor(edge_image).permute(2, 0, 1).unsqueeze(0).to("cuda", dtype=torch.float16)

        # Generate synthetic image with ControlNet
        synthetic_image = pipe(
            prompt=prompt,
            image=edge_image,
            strength=1.0,
            guidance_scale=7.5,
            num_inference_steps=50
        ).images[0]

        synthetic_image.save(os.path.join(synthetic_dir, f"synthetic_sitemap_{i}.png"))

print(f"Synthetic images saved in {synthetic_dir}")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Synthetic images saved in synthetic_sitemaps_v2


In [ ]:
import shutil
from google.colab import files

# Path to the folder you want to download
folder_path = '/content/synthetic_sitemaps_v2'

# Path to save the ZIP file
zip_path = '/content/synthetic_sitemaps_v2.zip'

# Compress the folder into a ZIP file
shutil.make_archive(base_name=folder_path, format='zip', root_dir=folder_path)

# Download the ZIP file
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>